General outline:
I want to take in an array (later image) representation of a scrabble board/hand tiles, and output the best play.

Naive best play: the highest scoring play

Goal best play: the play that maximizes your winning chances (takes into account the opponent's potential score)
    * You want to maximize how you're using the tiles in your hand. 
        * This doesn't just mean getting the highest score possible. Save your high tiles for the multipliers.
    * As well as minimizing your opponent's reply score

0. Constants:
    * Multiplier boards
    
* make functions that define the area that the word is in on the board (given a letter?)
    * allow another function to tell what the letters are in that area.


Main steps needed:
1. Take in an array of the board and your hand letters **check**
2. Tell the available permutations and their locations on the board **check**
    * return a dictionary with the word as a key and the location as the value **check**
3. See what crossing words those permutations are going to make **check**
    * this will return a dictionary of the main word and its position as a key and the crossing words as the values
    * could use a nested dictionary for this
4. Check to see if the permutations and their crossing words are in the dictionary
    * take in the dictinary from above?
    
For Step 2:
1. Need to get permutations for each row
       * Options:
           1. Get permutations around each group of letters like I've been doing
           2. Look at it on a row-wide basis
               * Need to make sure all the sample letters stay together
    

In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import csv
import itertools
import re
from collections import defaultdict


<IPython.core.display.Javascript object>

In [3]:
sample_board = [
    ["", "", "", "", "", "", "", "", "", "", "", "e", "", "", ""],  # 0
    ["", "", "", "", "", "", "", "", "", "z", "o", "n", "a", "", "w"],  # 1
    ["", "", "", "", "", "", "", "", "", "", "", "a", "", "", "i"],  # 2
    ["", "", "", "", "", "", "", "", "", "r", "i", "m", "a", "", "l"],  # 3
    ["", "", "", "", "", "", "", "", "", "", "", "o", "b", "e", "y"],  # 4
    ["", "", "", "", "", "", "", "", "", "j", "u", "r", "a", "l", ""],
    ["", "", "", "", "", "", "", "b", "e", "e", "p", "", "", "", ""],  # 6
    ["", "", "", "", "", "", "y", "e", "w", "s", "", "t", "", "", ""],
    ["", "", "", "", "", "", "", "l", "e", "t", "c", "h", "", "", ""],  # 8
    ["", "", "", "", "h", "u", "r", "l", "", "", "", "i", "", "r", ""],
    ["", "", "", "", "", "", "e", "", "", "", "", "g", "", "e", ""],  # 10
    ["", "", "", "", "", "", "d", "", "", "", "", "h", "u", "e", "d"],
    ["", "", "", "", "", "", "", "", "", "", "", "", "", "f", ""],  # 12
    ["", "", "", "", "", "", "", "", "", "v", "e", "e", "p", "s", ""],
    ["", "", "", "", "", "", "", "", "n", "u", "n", "", "", "", ""],  # 14
]
sample_board_letters = ["o", "d", "a", "s"]  # "d", "d", "a", "r"]

# numpy of the board
board_array = np.array(sample_board)

# making the empty strings spaces
neat_sample_array = np.copy(board_array)
neat_sample_array[neat_sample_array == ""] = " "


<IPython.core.display.Javascript object>

In [4]:
class Scrabbler:
    data_path = "../sowpods.txt"
    with open(data_path, newline="") as f:
        reader = csv.reader(f)
        allwords = list(reader)
    WORDLIST = list(itertools.chain.from_iterable(allwords))
    WORDLIST = [x.lower() for x in WORDLIST]
    WORDSET = set(WORDLIST)
    BOARD_LENGTH = 15
    MAX_I = 14

    LETTER_MULTIPLIERS = [
        [1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1],  # 0
        [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1],  # 1
        [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1],  # 2
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 3
        [1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1],  # 4
        [1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1],
        [3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3],  # 6
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [3, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 3],  # 8
        [1, 1, 1, 1, 1, 3, 1, 1, 1, 3, 1, 1, 1, 1, 1],
        [1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1],  # 10
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],
        [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1],  # 12
        [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1],
        [1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 1, 1, 1],  # 14
    ]
    LETTER_MULTIPLIERS_ARRAY = np.array(LETTER_MULTIPLIERS)

    WORD_MULTIPLIERS = [
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 0
        [1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1],  # 1
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 2
        [3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3],  # 3
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 4
        [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 6
        [1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 8
        [1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 10
        [3, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],  # 12
        [1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1],
        [1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1],  # 14
    ]
    WORD_MULTIPLIERS_ARRAY = np.array(WORD_MULTIPLIERS)

    TILE_SCORES = {
        "a": 1,
        "b": 4,
        "c": 4,
        "d": 2,
        "e": 1,
        "f": 4,
        "g": 3,
        "h": 3,
        "i": 1,
        "j": 10,
        "k": 5,
        "l": 2,
        "m": 4,
        "n": 2,
        "o": 1,
        "p": 4,
        "q": 10,
        "r": 1,
        "s": 1,
        "t": 1,
        "u": 2,
        "v": 5,
        "w": 4,
        "x": 8,
        "y": 3,
        "z": 10,
    }

    def __init__(self, board, hand_letters):
        self.board = board
        self.hand_letters = hand_letters
        self.T_board = np.copy(board).transpose()  # TODO: should this be here?

    # Navigator functions
    def check_yx(self, y, x):
        if x > -1 and x < 15 and y > -1 and y < 15:
            return bool(self.board[y, x])
        else:
            return "out"

    def check_right(self, y, x):
        if x < self.BOARD_LENGTH - 1:
            return bool(self.board[y, x + 1]), self.board[y, x + 1]
        else:
            return "out"

    def check_left(self, y, x):
        if x > 0:
            return bool(self.board[y, x - 1]), self.board[y, x - 1]
        else:
            return "out"

    def check_above(self, y, x):
        if y > 0:
            return bool(self.board[y - 1, x]), self.board[y - 1, x]
        else:
            return "out"

    def check_below(self, y, x):
        if y < self.BOARD_LENGTH - 1:
            return bool(self.board[y + 1, x]), self.board[y + 1, x]
        else:
            return "out"

    # For potential words of a given length
    # start at the left, travel right (keep track of empties) till you hit a letter
    def hand_letter_permutations(self, length):
        # this returns a list of permutations of the hand letters of a given length
        letter_permutations = list(itertools.permutations(self.hand_letters, length))
        return letter_permutations

    def row_permutations(self, row):
        y = row
        board_row = self.board[row]
        no_hand_letters = len(self.hand_letters)
        no_pre_filled = sum(list(map(bool, board_row)))
        row_perms_dict = {}
        if no_pre_filled > 0:
            for perm_len in range(1, no_hand_letters + 1):
                hand_perms = self.hand_letter_permutations(perm_len)
                # for x in range(0, BOARD_LENGTH-(len(self.hand_letters)-1)-no_pre_filled):
                # x=0
                no_tiles_to_right = no_pre_filled
                for hand_perm in hand_perms:
                    # while x + no_tiles_to_right+perm_len<self.BOARD_LENGTH:
                    x = 0
                    if (
                        self.check_yx(y, x) == False
                    ):  # and sum(list(map(bool,board_row[x:x+no_hand_letters])))
                        # for hand_perm in hand_perms:
                        while True:
                            # TODO check this
                            if (
                                any(
                                    board_row[max(0, x - 1) : min(x + perm_len + 1, 14)]
                                )
                                == True
                            ):
                                complete_word, word_y, word_x = self.complete_word_y_x(
                                    row, x, hand_perm
                                )
                                if complete_word == []:
                                    break
                                elif len(complete_word) > perm_len:
                                    # TODO: could leave out the part after the and below and delete duplicates later
                                    # TODO: could indicate the length of the original perm
                                    if (
                                        complete_word in row_perms_dict
                                        and [word_y, word_x]
                                        not in row_perms_dict[complete_word]
                                    ):
                                        row_perms_dict[complete_word].append(
                                            [word_y, word_x]
                                        )
                                    else:
                                        row_perms_dict[complete_word] = [
                                            [word_y, word_x]
                                        ]
                            x = x + 1
                            if x == self.BOARD_LENGTH:
                                break
                    # no_tiles_to_right=sum(list(map(bool,board_row[x+1:])))#TODO: +1 here?
        return row_perms_dict

    # fill the next len(hand_perm) spaces with hand tiles and find the connecting words
    # row=y
    def complete_word_y_x(self, row, x, hand_perm):
        hand_perm = list(hand_perm)
        perm_len = len(hand_perm)
        X = x
        board_row = self.board[row]
        y = row
        no_filled = 0
        filled_tiles = []
        left_letters = []
        right_letters = []
        while self.check_left(y, x)[0] == True:
            x = x - 1
            left_letters = left_letters + [board_row[x]]
        left_letters = left_letters[::-1]
        x = X
        while no_filled < perm_len:
            if self.check_yx(y, x) == True:
                filled_tiles = filled_tiles + [board_row[x]]
            elif self.check_yx(y, x) == "out":
                return [], [], []
            elif self.check_yx(y, x) == False:
                new_tile = hand_perm.pop(0)
                filled_tiles = filled_tiles + [new_tile]
                no_filled = no_filled + 1
            x = x + 1
        x = x - 1
        while self.check_right(y, x)[0] == True:
            x = x + 1
            right_letters = right_letters + [board_row[x]]
        complete_word_as_list = left_letters + filled_tiles + right_letters
        word_x = X - len(left_letters)
        complete_word = "".join(complete_word_as_list)
        return complete_word, y, word_x

    # Find the crossing words for a given complete_word_y_x info
    def find_crosses(self, complete_word, y, x):
        crossing_words = set()
        count = 0
        for tile in complete_word:
            # TODO: don't think I need this if part, only the last part and len(crossing_word)>1
            if (
                self.check_above(y, x)[0] == True or self.check_below(y, x)[0] == True
            ) and self.board[y, x] != tile:
                crossing_word, T_y, T_x = self.crossing_words_at_tile(tile, y, x)
                if len(crossing_word) > 1:
                    crossing_words.add((crossing_word, T_y, T_x))

            # TODO: not sure if I need all this stuff about edge_letter
            #             if self.board[y, x] != tile:
            #                 count = count + 1
            #                 edge_letter = tile
            #                 edge_x = x
            #                 edge_y = y

            x = x + 1
        # TODO fix this?
        #         if count == 1 and len(crossing_words) == 0:
        #             crossing_words.add((edge_letter, edge_y, edge_x))
        if len(crossing_words) == 0:
            crossing_words.add(("", -1, -1))
        return crossing_words

    # Enter a tile and its location and return the crosses that it makes
    def crossing_words_at_tile(self, tile, y, x):
        # We will be using the transpose board so y and x are switched
        self.board = self.board.transpose()
        y, x = x, y
        crossing_word, T_y, T_x = self.complete_word_y_x(y, x, (tile))
        self.board = self.board.transpose()
        return crossing_word, T_y, T_x

    # insert the "hook letter" into a shadow board and find the output connected to that
    # assume the board has already been transposed and you are operating with the right y and x
    def edge_perms(self, hook_letter, y, x):
        self.board[y, x] = hook_letter
        self.hand_letters.remove(hook_letter)
        shadow_row_perms = self.row_permutations(y)
        edge_perms_dict = {
            k: v
            for (k, v) in shadow_row_perms.items()
            if v[0][1] <= x
            and v[0][1] + len(k)
            >= x  # TODO: see if I really need to put out a nested list here
        }

        self.hand_letters.append(hook_letter)
        self.board[y, x] = ""
        return edge_perms_dict

    def find_valid_words(self):
        # perms_set = set()
        crossing_set = set()
        # board_perm_set=set()
        #board_perm_dict = defaultdict(list)
        board_perm_dict ={}
        
        #TODO: this block
        for row in range(0, self.BOARD_LENGTH):
            row_perms = self.row_permutations(row)
            # Probably going to need to combine dictionaries the old school way
            if (
                complete_word in row_perms_dict
                and [word_y, word_x]
                not in row_perms_dict[complete_word]
            ):
                row_perms_dict[complete_word].append(
                    [word_y, word_x]
                )
            else:
                row_perms_dict[complete_word] = [
                    [word_y, word_x]
                ]
            #End TODO
            
            for d in (board_perm_dict, row_perms):
                for key, value in d.items():
                    board_perm_dict[key].append(value)
        perms_set = set(dict(board_perm_dict).keys())
        valid_perms = self.WORDSET.intersection(perms_set)
        valid_perms_dict = {
            k: v for (k, v) in dict(board_perm_dict).items() if k in valid_perms
        }  # todo: work on the dictionary
        return valid_perms_dict
        # use the perms to find crossings

<IPython.core.display.Javascript object>

In [5]:
# for edge row, I could find permutations of the surrounding area including spaces
# make a check surrroundings function for both axes

<IPython.core.display.Javascript object>

In [6]:
sam = Scrabbler(board_array, sample_board_letters)
cw, _, _ = sam.complete_word_y_x(1, 1, ("a"))
sam.find_crosses("abcdfe", 12, 9)
# sam.edge_perms("s", 10, 7)

{('avu', 9, 12),
 ('ben', 10, 12),
 ('de', 14, 11),
 ('thighce', 11, 7),
 ('udp', 12, 11)}

<IPython.core.display.Javascript object>

In [ ]:
sam.find_valid_words()

In [14]:
print(np.where(board_array == "", " ", board_array))

[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'e' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'z' 'o' 'n' 'a' ' ' 'w']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'a' ' ' ' ' 'i']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'r' 'i' 'm' 'a' ' ' 'l']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'o' 'b' 'e' 'y']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'j' 'u' 'r' 'a' 'l' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' 'b' 'e' 'e' 'p' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'y' 'e' 'w' 's' ' ' 't' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' 'l' 'e' 't' 'c' 'h' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' 'h' 'u' 'r' 'l' ' ' ' ' ' ' 'i' ' ' 'r' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'e' ' ' ' ' ' ' ' ' 'g' ' ' 'e' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' 'd' ' ' ' ' ' ' ' ' 'h' 'u' 'e' 'd']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'f' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'v' 'e' 'e' 'p' 's' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'n' 'u' 'n' ' ' ' ' ' ' ' ']]


<IPython.core.display.Javascript object>

In [15]:
sam.complete_word_y_x(11, 6, ["a", "b", "c"])

('dabc', 11, 6)

<IPython.core.display.Javascript object>

In [9]:
a = sam.row_permutations(7)
#output: 
#{'dfdf':[[3,4]], 'dfkdf':[[5,7]],...}


<IPython.core.display.Javascript object>

In [ ]:
set(a.keys())

In [34]:
a = {"a", "b", "c"}
a.add("d", "e")

TypeError: add() takes exactly one argument (2 given)

<IPython.core.display.Javascript object>

In [59]:
d1 = {1: 2, 3: 4}
d2 = {1: 6, 3: 7}

dd = defaultdict(list)

for d in (d1, d2): # you can list as many input dicts as you want here
    for key, value in d.items():
        dd[key].append(value)

print(dict(dd).values())


dict_values([[2, 6], [4, 7]])


<IPython.core.display.Javascript object>

In [ ]:
            #             for key in row_perms.keys():
            #                 perms_set.add(key)
